In [1]:
import pandas as pd
import numpy as np
import re
import time
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, GRU, Embedding
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Input, Bidirectional
from keras.layers import Concatenate
tf.__version__

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'1.7.0'

In [2]:
import random
n_data = 1000
input_len = 5
output_len = 7
output_data = np.zeros((n_data,output_len),dtype=np.int32)
input_data = np.zeros((n_data, input_len), dtype = np.int32)
print output_data.shape
print input_data.shape
for i in range(n_data):
    for j in range(input_len):
        input_data[i][j] = random.randint(3,30)
        
for i in range(n_data):
    output_data[i][0] = 0
    output_data[i][6] = 30
    for j in range(1,6):
        output_data[i][j] = input_data[i][j-1]

        
print input_data[0]
print output_data[0]

(1000, 7)
(1000, 5)
[20 25 11  3 27]
[ 0 20 25 11  3 27 30]


In [3]:
vocab_size_input = 31
vocab_size_output = 31
encoder_seq_length = 5
decoder_seq_length = 7
latent_dim = 8
num_encoder_tokens = vocab_size_input
num_decoder_tokens = vocab_size_output
epochs = 10
batch_size = 128
total_data = 100000

In [4]:
np_encoder_input = input_data
np_decoder_input = output_data
decoder_input_data = np_decoder_input[:, :-1]
decoder_target_data = np_decoder_input[:, 1:]
encoder_input_data = np.copy(np_encoder_input)
print (decoder_input_data.shape)
print (encoder_input_data.shape)



(1000, 6)
(1000, 5)


In [5]:

print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)
max= 0 
for dataarray in decoder_target_data:
    for data in dataarray:
        if max < data:
            max=data

print max

(1000, 5)
(1000, 6)
(1000, 6)
30


In [6]:
import random
def generate_data(encoder_input_data,decoder_input_data,decoder_target_data):
    batch_size = 500
    re_encoder_input = np.zeros((500,5), dtype = np.int32)
    re_decoder_input = np.zeros((500,6), dtype= np.int32)
    re_decoder_target = np.zeros((500,6,31), dtype= np.int32)
    counter = 0
    max_i = len(encoder_input_data)
    while True:
        while counter < batch_size :
            i = random.randint(0,max_i)
            re_encoder_input[counter] = encoder_input_data[i][:]
            re_decoder_input[counter] = decoder_input_data[i][:]
            k = 0
            while k < len(decoder_target_data[i]):
                re_decoder_target[counter][k][decoder_target_data[i][k]] = 1
                k += 1
            counter += 1
        
        yield [re_encoder_input, re_decoder_input] , re_decoder_target

        



In [9]:
#latent_dim = 64
#num_encoder_tokens = 374
#num_decoder_tokens = 374
#epochs = 10
#batch_size = 64
encoder_inputs = Input(shape=(None,))
x = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
x, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(8, return_state=True))(x)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None,))
decoder_embedding_x = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
decoder_lstm = LSTM(16, return_sequences=True, return_state=True)

x,_,_ = decoder_lstm(decoder_embedding_x, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(x)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit_generator(generate_data(encoder_input_data, decoder_input_data, decoder_target_data),
          steps_per_epoch = 2,epochs = 100
          
          )

Epoch 1/100
2/2 [==============================] - 4s 2s/step - loss: 3.4316
Epoch 2/100
2/2 [==============================] - 1s 325ms/step - loss: 3.4251
Epoch 3/100
2/2 [==============================] - 1s 349ms/step - loss: 3.4193
Epoch 4/100
2/2 [==============================] - 1s 317ms/step - loss: 3.4132
Epoch 5/100
2/2 [==============================] - 1s 321ms/step - loss: 3.4067
Epoch 6/100
2/2 [==============================] - 1s 322ms/step - loss: 3.3995
Epoch 7/100
2/2 [==============================] - 1s 309ms/step - loss: 3.3915
Epoch 8/100
2/2 [==============================] - 0s 53ms/step - loss: 3.3824
Epoch 9/100
2/2 [==============================] - 0s 50ms/step - loss: 3.3721
Epoch 10/100
2/2 [==============================] - 0s 49ms/step - loss: 3.3603
Epoch 11/100
2/2 [==============================] - 0s 48ms/step - loss: 3.3468
Epoch 12/100
2/2 [==============================] - 0s 54ms/step - loss: 3.3313
Epoch 13/100
2/2 [===========================

In [24]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(16,))
decoder_state_input_c = Input(shape=(16,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding_x, initial_state=encoder_states)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

RuntimeError: Graph disconnected: cannot obtain value for tensor Tensor("input_5:0", shape=(?, ?), dtype=float32) at layer "input_5". The following previous layers were accessed without issue: []

In [16]:
encoder_model.summary()
decoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 8)      248         input_5[0][0]                    
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) [(None, 16), (None,  1088        embedding_5[0][0]                
__________________________________________________________________________________________________
concatenate_5 (Concatenate)     (None, 16)           0           bidirectional_3[0][1]            
                                                                 bidirectional_3[0][3]            
__________

In [36]:
def input_seq_to_num(sent):
    output_sent = np.zeros((1,20),dtype=np.int32)
    initial_list = []
    sent = clean_text(sent)
    for words in sent.split(' '):
        if words in vocab_to_int_input:
            initial_list.append(vocab_to_int_input[words])
        else:
            initial_list.append(vocab_to_int_input['<UNK>'])
            
    output_sent[0][0:len(initial_list)] = initial_list
    output_sent[0] = output_sent[0][::-1]
    return output_sent
    
#input_seq =  input_seq_to_num("hello how are you")
    

In [18]:
def speak(input_sent):
    states_value = encoder_model.predict(input_sent)
    encoder_input_seq =  np.zeros((1,1), dtype=np.int32)
    encoder_input_seq[0][0]  = 0
    stop_condition = False
    decoded_sent = []
    max_decoder_seq_length = 10
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([encoder_input_seq] + states_value)

        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        print sampled_token_index
        sampled_char = sampled_token_index
        decoded_sent.append(sampled_char)
        if (sampled_char == 30 or len(decoded_sent) > max_decoder_seq_length):
            stop_condition = True
        encoder_input_seq =  np.zeros((1,1), dtype=np.int32)
        encoder_input_seq[0][0]  = sampled_token_index
        states_value = [h, c]
    return 



In [13]:
print (speak(encoder_input_data[0]))
print (encoder_input_data[0])


11
11
11
11
11
11
11
11
11
11
11
None
[20 25 11  3 27]


In [19]:
print (speak([19 ,25  ,4  ,4  ,3 ,29 ,22  ,4 ,30  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0]))
print (speak([26 ,30 ,19 ,28 ,14 ,18 ,26  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0]))

10
10
30
None
11
11
30
None
